Assignment 5: Dario Antonio

In [1]:
import pygame
import moderngl
import glm
import numpy as np
from LoadObject import getObjectData

vertex_shader_code = """
#version 430 core
layout (location = 0) in vec3 in_position;
layout (location = 1) in vec3 in_normal;
uniform mat4 view;
uniform mat4 perspective;
out vec3 f_normal;

void main() {
    f_normal = in_normal;
    gl_Position = perspective * view * vec4(in_position, 1.0);
}
"""

fragment_shader_code = """
#version 430 core
in vec3 f_normal;
layout (location = 0) out vec4 out_color;

void main() {
    out_color = vec4(0.5 * normalize(f_normal) + 0.5, 1.0);
}
"""

pygame.init()
screen = pygame.display.set_mode((800, 600), pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption("Assignment 5: Dario Antonio")

gl = moderngl.create_context()

gl.enable(moderngl.DEPTH_TEST)
gl.clear_color = (0.0, 0.0, 0.0, 1.0)  

program = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code
)

objects = [
    'teapot.obj',
    '4_tetrahedron.obj',
    '6_hexahedron.obj',
    '8_octahedron.obj',
    '12_dodecahedron.obj',
    '20_icosahedron.obj',
]

def load_object(filename):
    obj_data, obj_bounds = getObjectData(filename, normal=True)
    return obj_data, obj_bounds

def compute_eye_position(obj_bounds):
    v = glm.vec3(glm.cos(glm.radians(45.0)), 1.0, glm.sin(glm.radians(45.0)))  
    return obj_bounds.center + 2.0 * obj_bounds.radius * v  

def compute_view_matrix(eye_position, obj_bounds):
    center = glm.vec3(obj_bounds.center[0], obj_bounds.center[1], obj_bounds.center[2])
    return glm.lookAt(eye_position, center, glm.vec3(0.0, 1.0, 0.0))  

def compute_perspective_matrix(obj_bounds, aspect_ratio):
    near_plane = 0.1 * obj_bounds.radius  
    far_plane = 5.0 * obj_bounds.radius  
    return glm.perspective(glm.radians(60.0), aspect_ratio, near_plane, far_plane)

def compute_model_matrix(bounds, angle):
    translate_matrix = glm.translate(glm.mat4(1), -glm.vec3(bounds.center))
    scale_matrix = glm.scale(glm.mat4(1), glm.vec3(1 / bounds.radius))
    rotate_matrix = glm.rotate(glm.mat4(1), angle, glm.vec3(0, 1, 0))  
    model_matrix = rotate_matrix * scale_matrix * translate_matrix
    return model_matrix

obj_data, obj_bounds = load_object(objects[0])

vertex_buffer = gl.buffer(obj_data.astype('f4').tobytes())
vao = gl.vertex_array(program, [(vertex_buffer, '3f 3f', 'in_position', 'in_normal')])

eye_position = compute_eye_position(obj_bounds)
aspect_ratio = screen.get_width() / screen.get_height()

running = True
clock = pygame.time.Clock()
current_obj_index = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_1:
                current_obj_index = 0
            elif event.key == pygame.K_2:
                current_obj_index = 1
            elif event.key == pygame.K_3:
                current_obj_index = 2
            elif event.key == pygame.K_4:
                current_obj_index = 3
            elif event.key == pygame.K_5:
                current_obj_index = 4
            elif event.key == pygame.K_6:
                current_obj_index = 5
            
            obj_data, obj_bounds = load_object(objects[current_obj_index])
            vertex_buffer = gl.buffer(obj_data.astype('f4').tobytes())
            vao = gl.vertex_array(program, [(vertex_buffer, '3f 3f', 'in_position', 'in_normal')])

            eye_position = compute_eye_position(obj_bounds)
            aspect_ratio = screen.get_width() / screen.get_height()

        if event.type == pygame.WINDOWRESIZED:
            aspect_ratio = event.x / event.y

    gl.clear()

    time_in_seconds = pygame.time.get_ticks() / 1000.0
    rotation_matrix = glm.rotate(glm.mat4(1.0), time_in_seconds, glm.vec3(0.0, 1.0, 0.0))

    rotated_eye_position = glm.vec3(rotation_matrix * glm.vec4(eye_position[0], eye_position[1], eye_position[2], 1.0))

    view_matrix = compute_view_matrix(rotated_eye_position, obj_bounds)
    perspective_matrix = compute_perspective_matrix(obj_bounds, aspect_ratio)
    model_matrix = compute_model_matrix(obj_bounds, time_in_seconds * 0.1)  

    program['view'].write(view_matrix)
    program['perspective'].write(perspective_matrix)

    vao.render(moderngl.TRIANGLES)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

pygame 2.6.0 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Normal exists
Normal exists
Normal exists
